In [9]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration, AutoConfig
from nnsight import LanguageModel
import nnsight
import torch
from IPython.display import clear_output

In [2]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
# model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to('cuda')
cfg = AutoConfig.from_pretrained("facebook/musicgen-small")

In [3]:
class MusicGenLanguageModel(LanguageModel):
    def _load_meta(
        self,
        repo_id: str,
        tokenizer_kwargs={},
        **kwargs,
    ):
        self.repo_id = repo_id

        self._load_config(repo_id, **kwargs)

        self._load_tokenizer(repo_id, **tokenizer_kwargs)
        return MusicgenForConditionalGeneration.from_pretrained(repo_id)

    def _load(
        self,
        repo_id: str,
        tokenizer_kwargs={},
        **kwargs,
    ):
        self.repo_id = repo_id

        self._load_config(repo_id, **kwargs)

        self._load_tokenizer(repo_id, **tokenizer_kwargs)
        return MusicgenForConditionalGeneration.from_pretrained(repo_id).to(kwargs["device_map"])


nn_model = MusicGenLanguageModel(
    "facebook/musicgen-small",
    config=cfg,
    tokenizer=processor.tokenizer,
    device_map="cuda",
)
with nn_model.generate("Hello world!", max_new_tokens=10):
    ...

clear_output()

In [4]:
nn_model.device

device(type='cuda', index=0)

In [5]:
nn_model

MusicgenForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): L

In [6]:
tokens = 255
prompt = "Recreate the essence of a classic video game theme with chiptune sounds and nostalgic melodies."
for n in [2, 8]:
    ablate_layer = nn_model.decoder.model.decoder.layers[n]
    with nn_model.generate([prompt] * 3, max_new_tokens=tokens):
        outputs = nnsight.list().save()  # Initialize & .save() nnsight list
        for _ in range(tokens):
            ablate_layer.output[0][:] = ablate_layer.input[0][:]
            outputs.append(nn_model.generator.output)
            nn_model.next()
    # torchaudio.save(
    #     f"out_{n}.wav",
    #     src=outputs[0][0].detach().cpu(),
    #     sample_rate=nn_model.config.sampling_rate,
    #     channels_first=True,
    # )

In [7]:
outputs[0].shape

torch.Size([3, 1, 161280])

In [ ]:
nn_model.cpu()
del nn_model

NameError: name 'nn_model' is not defined

In [11]:
torch.cuda.empty_cache()